In [ ]:
# syft absolute
import syft as sy
from syft.service.log.log import SyftLogV3
from syft.types.blob_storage import BlobStorageEntry
from syft.types.blob_storage import CreateBlobStorageEntry
from syft.types.syft_object import Context
from syft.types.syft_object import SyftObject

In [ ]:
print(f"syft version: {sy.__version__}")

TODOS
- [x] action objects
- [x] maybe an example of how to migrate one object type in a custom way
- [x] check SyftObjectRegistry and compare with current implementation
- [x] run unit tests
- [ ] finalize notebooks for testing, run in CI
- [ ] other tasks defined in tickets
- [ ] also get actionobjects in get_migration_objects
- [ ] make clientside method to migrate blobstorage `migrate_blobstorage(from_client, to_client, migration_data)`
- [ ] make domainclient get_migration and apply_migration (to/from file?)
    - merge with save_migration_objects_to_file / migrate_objects_from_file 

In [ ]:
node = sy.orchestra.launch(
    name="test_upgradability",
    dev_mode=True,
    local_db=True,
    n_consumers=2,
    create_producer=True,
    migrate=False,
)

In [ ]:
client = node.login(email="info@openmined.org", password="changethis")

# Client side migrations

In [ ]:
temp_node = sy.orchestra.launch(
    name="temp_node",
    dev_mode=True,
    local_db=True,
    n_consumers=2,
    create_producer=True,
    migrate=False,
    reset=True,
)

temp_client = temp_node.login(email="info@openmined.org", password="changethis")

In [ ]:
migration_data = client.services.migration.get_migration_data()

In [ ]:
# syft absolute
from syft.client.migrations import migrate_blob_storage

In [ ]:
sy.migrate(from_client=client, to_client=temp_client)

In [ ]:
migrate_blob_storage(
    from_client=client,
    to_client=temp_client,
    blob_storage_objects=migration_data.blob_storage_objects,
)

In [ ]:
migration_data.store_objects

In [ ]:
temp_client.api.services.migration.apply_migration_data(migration_data)

## document store objects

In [ ]:
migration_dict = client.services.migration.get_migration_objects(get_all=True)

In [ ]:
migration_dict

In [ ]:
def custom_migration_function(context, obj: SyftObject, klass) -> SyftObject:
    # Here, we are just doing the same, but this is where you would write your custom logic
    return obj.migrate_to(klass.__version__, context)

In [ ]:
# this wont work in the cases where the context is actually used,
# but since this would need custom logic here anyway you write workarounds for that (manually querying required state)


context = Context()
migrated_objects = []
for klass, objects in migration_dict.items():
    for obj in objects:
        if isinstance(obj, BlobStorageEntry):
            continue
        elif isinstance(obj, SyftLogV3):
            migrated_obj = custom_migration_function(context, obj, klass)
        else:
            try:
                migrated_obj = obj.migrate_to(klass.__version__, context)
            except Exception:
                print(obj.__version__, obj.__canonical_name__)
                print(klass.__version__, klass.__canonical_name__)
                raise

        migrated_objects.append(migrated_obj)

In [ ]:
# TODO what to do with workerpools
# TODO what to do with admin? @yash: can we make new node with existing verifykey?
# TODO check asset AO is not saved in blobstorage

In [ ]:
res = temp_client.services.migration.create_migrated_objects(migrated_objects)

In [ ]:
res

In [ ]:
assert isinstance(res, sy.SyftSuccess)

# Migrate blobstorage

In [ ]:
klass = BlobStorageEntry
blob_entries = migration_dict[klass]
obj = blob_entries[0]
obj

In [ ]:
# stdlib
from io import BytesIO
import sys


def migrate_blob_entry_data(
    old_client, new_client, obj, klass
) -> sy.SyftSuccess | sy.SyftError:
    migrated_obj = obj.migrate_to(klass.__version__, Context())
    uploaded_by = migrated_obj.uploaded_by
    blob_retrieval = old_client.services.blob_storage.read(obj.id)
    if isinstance(blob_retrieval, sy.SyftError):
        return blob_retrieval

    data = blob_retrieval.read()
    # TODO do we have to determine new filesize here?
    serialized = sy.serialize(data, to_bytes=True)
    size = sys.getsizeof(serialized)
    blob_create = CreateBlobStorageEntry.from_blob_storage_entry(obj)
    blob_create.file_size = size

    blob_deposit_object = new_client.services.blob_storage.allocate_for_user(
        blob_create, uploaded_by
    )
    if isinstance(blob_deposit_object, sy.SyftError):
        return blob_deposit_object
    return blob_deposit_object.write(BytesIO(serialized))

In [ ]:
for blob_entry in blob_entries:
    res = migrate_blob_entry_data(client, temp_client, blob_entry, BlobStorageEntry)
    display(res)

In [ ]:
client.services.blob_storage.get_all()

## Actions and ActionObjects

In [ ]:
migration_action_dict = client.services.migration.get_migration_actionobjects(
    get_all=True
)

In [ ]:
ao = migration_action_dict[list(migration_action_dict.keys())[0]][0]

In [ ]:
ao.syft_action_data_cache

In [ ]:
node.python_node.action_store.data

In [ ]:
client.jobs[0].result.id.id

In [ ]:
node.python_node.action_store.data[sy.UID("106b561961c74a46afc63c5c73c24212")].__dict__

In [ ]:
# this wont work in the cases where the context is actually used, but since this you would need custom logic here anyway
# it doesnt matter
context = Context()
migrated_actionobjects = []
for klass, objects in migration_action_dict.items():
    for obj in objects:
        # custom migration logic here
        migrated_actionobject = obj.migrate_to(klass.__version__, context)
        migrated_actionobjects.append(migrated_actionobject)

In [ ]:
print(migrated_actionobjects)

In [ ]:
res = temp_client.services.migration.update_migrated_actionobjects(
    migrated_actionobjects
)

In [ ]:
res

In [ ]:
assert isinstance(res, sy.SyftSuccess)

In [ ]:
for uid in temp_node.python_node.action_store.data:
    ao = temp_client.services.action.get(uid)
    ao.reload_cache()
    print(ao.syft_action_data_cache)

In [ ]:
for uid in node.python_node.action_store.data:
    ao = client.services.action.get(uid)
    ao.reload_cache()
    print(ao.syft_action_data_cache)

## Store metadata

- Permissions
- StoragePermissions

In [ ]:
store_metadata = client.services.migration.get_all_store_metadata()
store_metadata

In [ ]:
for k, v in store_metadata.items():
    if len(v.permissions):
        print(
            k, len(v.permissions), len(v.permissions) == len(migration_dict.get(k, []))
        )

In [ ]:
# Test update method with a temp node
# After update, all metadata should match between the nodes

In [ ]:
temp_client.services.migration.update_store_metadata(store_metadata)

In [ ]:
for cname, real_partition in node.python_node.document_store.partitions.items():
    temp_partition = temp_node.python_node.document_store.partitions[cname]

    temp_perms = dict(temp_partition.permissions.items())
    real_perms = dict(real_partition.permissions.items())

    for k, temp_v in temp_perms.items():
        if k not in real_perms:
            continue
        real_v = real_perms[k]
        assert real_v.issubset(temp_v)

    temp_storage = dict(temp_partition.storage_permissions.items())
    real_storage = dict(real_partition.storage_permissions.items())
    for k, temp_v in temp_storage.items():
        if k not in real_storage:
            continue
        real_v = real_storage[k]
        assert real_v.issubset(temp_v)

# Action store
real_partition = node.python_node.action_store
temp_partition = temp_node.python_node.action_store
temp_perms = dict(temp_partition.permissions.items())
real_perms = dict(real_partition.permissions.items())

# Only look at migrated items
temp_perms = {k: v for k, v in temp_perms.items() if k in real_perms}
for k, temp_v in temp_perms.items():
    if k not in real_perms:
        continue
    real_v = real_perms[k]
    assert real_v.issubset(temp_v)

temp_storage = dict(temp_partition.storage_permissions.items())
real_storage = dict(real_partition.storage_permissions.items())
for k, temp_v in temp_storage.items():
    if k not in real_storage:
        continue
    real_v = real_storage[k]
    assert real_v.issubset(temp_v)